<img src="images/logos/The-Construct-logo-new.png" width="700">

# PRESENTS...

# ROS Developers Open Class n.148

<img src="images/Open-Class-148.jpg" width="650" />

## How to Launch the Simulation

To start the simulation, first we'll need to source our workspace:

- Open a terminal window by clicking on the shell icon on the bottom left side of your screen:

<img src="images/shell-superapp.png" width="450" />

- Copy and paste the following terminal commands in your shell:


<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in Shell
</span>

In [ ]:
source ~/simulation_ws/devel/setup.bash
roslaunch ur_e_gazebo ur3e.launch

And that's it! You should be able to see the simulation and control everything as if it was the real robot if you go to the Gazebo button in the bottom left side of your screen:

<img src="images/gazebo-icon.png" width="100" />


**Wait around 30 seconds maximum** for the simulaion to start and you should see this simulation now:


<img src="images/ur3e-sim.png" width="800" />

## Requirements

<a href="https://app.theconstructsim.com/Course/81" target="_blank">ROS2 Manipulation Basics</a>

<a href="https://app.theconstructsim.com/#/Course/73" target="_blank"><img src="images/ros2-manip.png" width="" /></a>

<div class="jumbotron m-0">
    <hr />
    <h1 class="text-center">
        <span class="text-primary">
            ROS2 Motion Planning with C++
        </span>
    </h1>
    <hr />
</div>

## Test Moveit2

Before actually starting with the class, let's check that we can run Moveit2 correctly. For this, you'll need 3 steps:

### 1. Launch the parameter bridge

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in Shell
</span>

In [ ]:
source ~/catkin_ws/devel/setup.bash
roslaunch load_params load_params.launch
source /home/simulations/ros2_sims_ws/install/setup.bash
ros2 run ros1_bridge parameter_bridge __name:=parameter_bridge

### 2. Launch the action bridge

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in Shell
</span>

In [ ]:
source /opt/ros/noetic/setup.bash
source /home/simulations/ros2_sims_ws/install/setup.bash
ros2 launch start_bridge start_bridge.launch.py

### 3. Launch Moveit2

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in Shell
</span>

In [ ]:
source ~/ros2_ws/install/setup.bash
ros2 launch my_moveit2_config my_planning_execution.launch.py launch_rviz:=true

## Move Group C++ Interface

In the previous section, you could plan and execute trajectories for your robot using the MoveIt2 RVIZ environment. But, this is not the typical case.

Usually, you will want to move your robot with your own scripts. And this is exactly what you are going to do now! However, you will use C++ to control the robot.

### Planning a trajectory

As you've seen in the previous example, **there is a difference between planning and executing a trajectory**. So, in this first part, you will see how to plan a trajectory with C++. For that, complete the next exercise.

<div class="bg-success text-center">
    - Exercise 1 -
</div>

Let's have a quick look at the code that you will be executing:

<span class="badge badge-pill badge-primary">
    <i class="fa fa-file"></i>
    &nbsp;
    test_trajectory.cpp
</span>

In [ ]:
#include <moveit/move_group_interface/move_group_interface.h>
#include <moveit/planning_scene_interface/planning_scene_interface.h>

#include <moveit_msgs/msg/display_robot_state.hpp>
#include <moveit_msgs/msg/display_trajectory.hpp>

#include <moveit_msgs/msg/attached_collision_object.hpp>
#include <moveit_msgs/msg/collision_object.hpp>

#include "rclcpp/rclcpp.hpp"
#include <string>

static const rclcpp::Logger LOGGER = rclcpp::get_logger("move_group_demo");
static const std::string PLANNING_GROUP_ARM = "ur_manipulator";

class TestTrajectory : public rclcpp::Node {
public:
  TestTrajectory(std::shared_ptr<rclcpp::Node> move_group_node)
      : Node("test_trajectory"),
        move_group_arm(move_group_node, PLANNING_GROUP_ARM),
        joint_model_group_arm(
            move_group_arm.getCurrentState()->getJointModelGroup(
                PLANNING_GROUP_ARM)) {

    this->timer_ =
        this->create_wall_timer(std::chrono::milliseconds(500),
                                std::bind(&TestTrajectory::timer_callback, this));

  } // end of constructor

  // Getting Basic Information
  void get_info() {

    RCLCPP_INFO(LOGGER, "Planning frame: %s", move_group_arm.getPlanningFrame().c_str());
    RCLCPP_INFO(LOGGER, "End-effector link: %s", move_group_arm.getEndEffectorLink().c_str());
    RCLCPP_INFO(LOGGER, "Available Planning Groups:");
    std::copy(move_group_arm.getJointModelGroupNames().begin(), move_group_arm.getJointModelGroupNames().end(),
                std::ostream_iterator<std::string>(std::cout, ", "));
  
  }

  void current_state() {
    RCLCPP_INFO(LOGGER, "Get Robot Current State");

    current_state_arm = move_group_arm.getCurrentState(10);

    current_state_arm->copyJointGroupPositions(this->joint_model_group_arm,
                                               this->joint_group_positions_arm);
  }

  void plan_arm_joint_space() {

    RCLCPP_INFO(LOGGER, "Planning to Joint Space");

    //joint_group_positions_arm[0] = 0.00;  // Shoulder Pan
    joint_group_positions_arm[1] = -2.50;  // Shoulder Lift
    joint_group_positions_arm[2] = 1.50;  // Elbow
    joint_group_positions_arm[3] = -1.50;  // Wrist 1
    joint_group_positions_arm[4] = -1.55;  // Wrist 2
    //joint_group_positions_arm[5] = 0.00;  // Wrist 3

    move_group_arm.setJointValueTarget(joint_group_positions_arm);

    bool success_arm = (move_group_arm.plan(my_plan_arm) ==
                        moveit::planning_interface::MoveItErrorCode::SUCCESS);

  }

  // Timer Callback function
  void timer_callback() {

    this->timer_->cancel();
    get_info();
    current_state();
    plan_arm_joint_space();
  }

private:
  moveit::planning_interface::PlanningSceneInterface planning_scene_interface;
  std::vector<double> joint_group_positions_arm;
  moveit::planning_interface::MoveGroupInterface::Plan my_plan_arm;
  rclcpp::TimerBase::SharedPtr timer_;

  moveit::planning_interface::MoveGroupInterface move_group_arm;

  const moveit::core::JointModelGroup *joint_model_group_arm;

  moveit::core::RobotStatePtr current_state_arm;

}; // End of Class

int main(int argc, char **argv) {
  rclcpp::init(argc, argv);
  rclcpp::NodeOptions node_options;
  node_options.automatically_declare_parameters_from_overrides(true);
  auto move_group_node =
      rclcpp::Node::make_shared("move_group_demo", node_options);

  rclcpp::executors::SingleThreadedExecutor planner_executor;
  std::shared_ptr<TestTrajectory> planner_node =
      std::make_shared<TestTrajectory>(move_group_node);
  planner_executor.add_node(planner_node);
  planner_executor.spin();

  rclcpp::shutdown();
  return 0;
}

That's great! But, how does this code work? What does each part mean? Let's break it down into smaller pieces.

First of all we define a couple of static variables for the **logger** system and for the **planning group of our arm**, which is named **`ur_manipulator`**:

In [ ]:
static const rclcpp::Logger LOGGER = rclcpp::get_logger("move_group_demo");
static const std::string PLANNING_GROUP_ARM = "ur_manipulator";

We define a class named **`TestTrajectory`**, which inherits from **`Node`**:

In [ ]:
class TestTrajectory : public rclcpp::Node

Next we find the constructor of the class:

In [ ]:
TestTrajectory(std::shared_ptr<rclcpp::Node> move_group_node)
  : Node("pick_and_place"),
    move_group_arm(move_group_node, PLANNING_GROUP_ARM),
    joint_model_group_arm(
        move_group_arm.getCurrentState()->getJointModelGroup(
            PLANNING_GROUP_ARM)) {

this->timer_ =
    this->create_wall_timer(std::chrono::milliseconds(500),
                            std::bind(&TestTrajectory::timer_callback, this));

}

As you can see, we need to initialize in the constructor 3 things:

* A **`Node`** object.


* A **`MoveGroupInterface`** object, in this case **move_group_arm**.


* A **`JointModelGroup`** object, in this case **joint_model_group_arm**.

Let's pay special attention to the `MoveGroupInterface`, which allows you to communicate with MoveIt2. It can be set up using just the name of the planning group you would like to control and plan for.

Finally, also inside the constructor, we create a Timer object **`timer_`**.

Next we find the **`get_info()`** method:

In [ ]:
void get_info() {

    RCLCPP_INFO(LOGGER, "Planning frame: %s", move_group_arm.getPlanningFrame().c_str());
    RCLCPP_INFO(LOGGER, "End-effector link: %s", move_group_arm.getEndEffectorLink().c_str());
    RCLCPP_INFO(LOGGER, "Available Planning Groups:");
    std::copy(move_group_arm.getJointModelGroupNames().begin(), move_group_arm.getJointModelGroupNames().end(),
                std::ostream_iterator<std::string>(std::cout, ", "));

}

Here we get some data about our MoveIt2 setup. This is not mandatory for the planning, but it can be very useful to make sure everything is configured as it should. Specifically, you are getting data about:

* The name of the frame in which the robot is planning

* The current end-effector link

* The available Planning Groups

Next we find the **`current_state()`** method:

In [ ]:
void current_state() {
    RCLCPP_INFO(LOGGER, "Get Robot Current State");

    current_state_arm = move_group_arm.getCurrentState(10);

    current_state_arm->copyJointGroupPositions(this->joint_model_group_arm,
                                               this->joint_group_positions_arm);
}

What we are doing here is to get the current state of the robot. Then we copy the values of the variable **`joint_model_group_arm`** (which contains the position value of each joint) into the variable **`joint_group_positions_arm`**.

Next we find the **`plan_arm_joint_space()`** method:

In [ ]:
void plan_arm_joint_space() {

    RCLCPP_INFO(LOGGER, "Planning to Joint Space");

    //joint_group_positions_arm[0] = 0.00;  // Shoulder Pan
    joint_group_positions_arm[1] = -2.50;  // Shoulder Lift
    joint_group_positions_arm[2] = 1.50;  // Elbow
    joint_group_positions_arm[3] = -1.50;  // Wrist 1
    joint_group_positions_arm[4] = -1.55;  // Wrist 2
    //joint_group_positions_arm[5] = 0.00;  // Wrist 3

    move_group_arm.setJointValueTarget(joint_group_positions_arm);

    bool success_arm = (move_group_arm.plan(my_plan_arm) ==
                        moveit::planning_interface::MoveItErrorCode::SUCCESS);

}

Here we are doing a couple of important things:

* First, we set the desired position values of the joints. In this case, we are setting the joint values of the **home** position.


* Second, we set these new values as the new target using the **`setJointValueTarget`** method.


* Finally, we call the **`plan()`** method of the move group interface to plan the trajectory.

Finally we have the **`timmer_callback()`** method:

In [ ]:
void timer_callback() {

    this->timer_->cancel();
    get_info();
    current_state();
    plan_arm_joint_space();
  }

Here we just call the different methods previously defined to perform the planning pipeline. Also, we cancel the timer so that it's only executed 1 time.

At the end of the class you can find the definition of all the different variables that are needed:

In [ ]:
private:
  moveit::planning_interface::PlanningSceneInterface planning_scene_interface;
  std::vector<double> joint_group_positions_arm;
  moveit::planning_interface::MoveGroupInterface::Plan my_plan_arm;
  rclcpp::TimerBase::SharedPtr timer_;

  moveit::planning_interface::MoveGroupInterface move_group_arm;

  const moveit::core::JointModelGroup *joint_model_group_arm;

  moveit::core::RobotStatePtr current_state_arm;

};

Finally we have the **`main()`** function:

In [ ]:
int main(int argc, char **argv) {
  rclcpp::init(argc, argv);
  rclcpp::NodeOptions node_options;
  node_options.automatically_declare_parameters_from_overrides(true);
  auto move_group_node =
      rclcpp::Node::make_shared("move_group_demo", node_options);

  rclcpp::executors::SingleThreadedExecutor planner_executor;
  std::shared_ptr<TestTrajectory> planner_node =
      std::make_shared<TestTrajectory>(move_group_node);
  planner_executor.add_node(planner_node);
  planner_executor.spin();

  rclcpp::shutdown();
  return 0;
}

Here we are creating a ROS2 node and adding this node to a `SingleThreadedExecutor` executor. Then, we spin this executor until somebody terminates the program.

You can have a deeper look at the functions provided by `MoveGroupInterface` here: http://docs.ros.org/en/noetic/api/moveit_ros_planning_interface/html/classmoveit_1_1planning__interface_1_1MoveGroupInterface.html

### Let's create the files and launch them

a) Inside your **ros2_ws**, create a new package named **moveit2_scripts** with some dependencies.

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in Shell #1
</span>

In [ ]:
source /opt/ros/foxy/setup.bash
cd ~/ros2_ws/src
ros2 pkg create moveit2_scripts --dependencies rclcpp rclcpp_action moveit_core moveit_ros_planning moveit_ros_planning_interface interactive_markers geometric_shapes control_msgs moveit_msgs

Inside this package, create a new file named **test_trajectory.cpp** and copy the code you've just seen above inside this file.

In [ ]:
cd ~/ros2_ws/src/moveit2_scripts/src
touch test_trajectory.cpp

b) Inside the package, also create a launch file to launch the program, called **test_trajectory.launch.py**.

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in Shell #1
</span>

In [ ]:
cd ~/ros2_ws/src/moveit2_scripts
mkdir launch
touch launch/test_trajectory.launch.py

Paste the code below into this launch file:

<span class="badge badge-pill badge-primary">
    <i class="fa fa-file"></i>
    &nbsp;
    test_trajectory.launch.py
</span>

In [ ]:
import os
import yaml
from launch import LaunchDescription
from launch.actions import DeclareLaunchArgument
from launch.substitutions import Command, FindExecutable, LaunchConfiguration, PathJoinSubstitution
from launch_ros.actions import Node
from launch_ros.substitutions import FindPackageShare
from ament_index_python.packages import get_package_share_directory


def load_file(package_name, file_path):
    package_path = get_package_share_directory(package_name)
    absolute_file_path = os.path.join(package_path, file_path)

    try:
        with open(absolute_file_path, "r") as file:
            return file.read()
    except EnvironmentError:  # parent of IOError, OSError *and* WindowsError where available
        return None

def load_file2(file_path):
    """Load the contents of a file into a string"""
    try:
        with open(file_path, 'r') as file:
            return file.read()
    except EnvironmentError: # parent of IOError, OSError *and* WindowsError where available
        return None


def load_yaml(package_name, file_path):
    package_path = get_package_share_directory(package_name)
    absolute_file_path = os.path.join(package_path, file_path)

    try:
        with open(absolute_file_path, "r") as file:
            return yaml.safe_load(file)
    except EnvironmentError:  # parent of IOError, OSError *and* WindowsError where available
        return None

def get_package_file(package, file_path):
    """Get the location of a file installed in an ament package"""
    package_path = get_package_share_directory(package)
    absolute_file_path = os.path.join(package_path, file_path)
    return absolute_file_path


def generate_launch_description():

    declared_arguments = []
    # UR specific arguments
    declared_arguments.append(
        DeclareLaunchArgument(
            "safety_limits",
            default_value="true",
            description="Enables the safety limits controller if true.",
        )
    )
    declared_arguments.append(
        DeclareLaunchArgument(
            "safety_pos_margin",
            default_value="0.15",
            description="The margin to lower and upper limits in the safety controller.",
        )
    )
    declared_arguments.append(
        DeclareLaunchArgument(
            "safety_k_position",
            default_value="20",
            description="k-position factor in the safety controller.",
        )
    )
    # General arguments
    declared_arguments.append(
        DeclareLaunchArgument(
            "description_package",
            default_value="ur_e_description",
            description="Description package with robot URDF/XACRO files. Usually the argument \
        is not set, it enables use of a custom description.",
        )
    )
    declared_arguments.append(
        DeclareLaunchArgument(
            "description_file",
            default_value="ur3e_robot.urdf.xacro",
            description="URDF/XACRO description file with the robot.",
        )
    )
    declared_arguments.append(
        DeclareLaunchArgument(
            "moveit_config_package",
            default_value="my_moveit2_config",
            description="MoveIt config package with robot SRDF/XACRO files. Usually the argument \
        is not set, it enables use of a custom moveit config.",
        )
    )
    declared_arguments.append(
        DeclareLaunchArgument(
            "moveit_config_file",
            default_value="ur3e.srdf",
            description="MoveIt SRDF/XACRO description file with the robot.",
        )
    )
    declared_arguments.append(
        DeclareLaunchArgument(
            "prefix",
            default_value='""',
            description="Prefix of the joint names, useful for \
        multi-robot setup. If changed than also joint names in the controllers' configuration \
        have to be updated.",
        )
    )
    declared_arguments.append(
        DeclareLaunchArgument("launch_rviz", default_value="true", description="Launch RViz?")
    )

    # Initialize Arguments
    safety_limits = LaunchConfiguration("safety_limits")
    safety_pos_margin = LaunchConfiguration("safety_pos_margin")
    safety_k_position = LaunchConfiguration("safety_k_position")
    # General arguments
    description_package = LaunchConfiguration("description_package")
    description_file = LaunchConfiguration("description_file")
    moveit_config_package = LaunchConfiguration("moveit_config_package")
    moveit_config_file = LaunchConfiguration("moveit_config_file")
    prefix = LaunchConfiguration("prefix")
    launch_rviz = LaunchConfiguration("launch_rviz")

    robot_description_content = Command(
        [
            PathJoinSubstitution([FindExecutable(name="xacro")]),
            " ",
            PathJoinSubstitution([FindPackageShare(description_package), "urdf", description_file]),
            " ",
            "safety_limits:=",
            safety_limits,
            " ",
            "safety_pos_margin:=",
            safety_pos_margin,
            " ",
            "safety_k_position:=",
            safety_k_position,
            " ",
            "name:=",
            # Also, ur_type parameter could be used, but then the planning group names in YAML
            # configs have to be updated!
            "ur",
            " ",
            "prefix:=",
            prefix,
            " ",
        ]
    )
    robot_description = {"robot_description": robot_description_content}

    # MoveIt Configuration
    srdf_file = load_file('my_moveit2_config', 'config/ur3e.srdf')
    robot_description_semantic = {"robot_description_semantic": srdf_file}

    kinematics_yaml = load_yaml(
        "my_moveit2_config", "config/kinematics.yaml"
    )

    # MoveGroupInterface demo executable
    move_group_demo = Node(
        name="test_trajectory",
        package="moveit2_scripts",
        executable="test_trajectory",
        output="screen",
        parameters=[
            {'use_sim_time': True},
            robot_description,
            robot_description_semantic, 
            kinematics_yaml,
            ],
    )

    nodes_to_start = [move_group_demo]

    return LaunchDescription(declared_arguments + nodes_to_start)

As you can see, the launch file is very similar to the one used for launching MoveIt2. You are loading the same parameters, and the only difference is that you are now launching our C++ script. Also, it's unnecessary to start RVIZ this time since you already have it running.

In [ ]:
move_group_demo = Node(
    name="test_trajectory",
    package="moveit2_scripts",
    executable="test_trajectory",
    output="screen",
    parameters=[
        {'use_sim_time': True},
        robot_description,
        robot_description_semantic, 
        kinematics_yaml,
        ],
)

c) Now, update the **CMakeLists.txt** file to compile everything. First, open the file and add the following lines to the code:

<span class="badge badge-pill badge-primary">
    <i class="fa fa-file"></i>
    &nbsp;
    Add to CMakeLists.txt
</span>

In [ ]:
# Generate the executable
add_executable(test_trajectory
    src/test_trajectory.cpp)
target_include_directories(test_trajectory
    PUBLIC include)
ament_target_dependencies(test_trajectory
    ament_cmake
	rclcpp
	rclcpp_action
	moveit_core
	moveit_ros_planning_interface
	interactive_markers
	moveit_ros_planning
	control_msgs)

# Install the executable
install(TARGETS 
	test_trajectory
    DESTINATION lib/${PROJECT_NAME}
)

# Install the launch file
install(DIRECTORY 
	launch
    DESTINATION share/${PROJECT_NAME}
)

You are generating an executable from the file **test_trajectory.cpp** and installing it alongside the contents of the launch folder.

d) Great! Let's now compile our workspace.

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in Shell #1
</span>

In [ ]:
cd ~/ros2_ws
colcon build
source install/setup.bash

e) Now test your new program! First, you'll need to launch the MoveIt2 RVIZ environment.

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in Shell #1
</span>

In [ ]:
ros2 launch my_moveit2_config my_planning_execution.launch.py launch_rviz:=true

f) Now, run your code:

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in Shell #4
</span>

In [ ]:
source ~/ros2_ws/install/setup.bash
ros2 launch moveit2_scripts test_trajectory.launch.py

You will see how the plan to the **home** position is computed.

<img src="images/moveit2-joint-plan.gif" width="600" />

<div class="bg-success text-center">
    - End of Exercise 1 -
</div>

### Executing a trajectory

So, at this point, you've seen some methods that allow you to plan a trajectory with C++ code. But what about executing this trajectory? 

It's very simple. You need to call the `execute (plan)` function from the planning group to execute a trajectory. For instance, given a plan `my_plan`, you would execute the planned motion like this:

In [ ]:
move_group_arm.execute(my_plan_arm);

By executing this line of code, you will be telling your robot to **Execute** the trajectory stored in the `my_plan` variable.

Also, there's the option to **Plan & Execute** a motion, just as the GUI provides. 

In [ ]:
move_group_arm.move();

<div class="bg-success text-center">
    - Exercise 2 -
</div>

Modify the previous code so that now, it also executes the trajectory. You should see the trajectory executed in the simulation, like shown below:

<img src="images/moveit-execute.gif" width="600" />

<div class="bg-success text-center">
    - End of Exercise 2 -
</div>

## Remember you can learn more about ROS2 Manipulation in our course in the Academy:

<a href="https://app.theconstructsim.com/Course/81" target="_blank">ROS2 Manipulation Basics</a>

<a href="https://app.theconstructsim.com/#/Course/73" target="_blank"><img src="images/ros2-manip.png" width="" /></a>

# Build the future, Become a ROS DEVELOPER!